## Python-for-Lunch on defensive programming

In this tutorial, we will be introducting the concept of **Defensive Programming**, the main idea of which is to constantly check if your code is doing what you expect it to do. The most important concept is that of **unit tests**, which are small, often almost trivial, tests (using `assert` statements in python) that catch anything unexpected.

In our [Parcels framework](https://github.com/OceanParcels/parcels) we have hundreds of these unit tests, and this makes development much easier. When we add functionality, we can assume we haven't broken Parcels somewhere else if all the unit tests still work (assuming that our unit tests cover _all_ existing functionality, which of course is a big assumption).

Here, we will use create some simple functions to show how to use `assert` statements for unit testing, and also introduce the concept of **Test driven development**.

In [ ]:
import numpy as np

The following function calculates the Coriolis parameter as a function of latitude

In [ ]:
def calculate_coriolis(lat):
    omega = 2*np.pi/86400.
    return 2*omega*np.sin(lat/180.*np.pi)

calculate_coriolis(45)

However, the Coriolis parameter is only defined for latitudes between 90S and 90N. Our function, however, happily calculates the Coriolis parameter for clearly non-sensical latitudes

In [ ]:
calculate_coriolis(128480)

The solution is to check (using the `assert` command) whether the input is between 90S and 90N, and throw an error if that is not the case

In [ ]:
def calculate_coriolis(lat):
    assert -90 < lat < 90, 'lat should be between -90 and 90'
    omega = 2*np.pi/86400.
    return 2*omega*np.sin(lat/180.*np.pi)

calculate_coriolis(180)

### Test-Driven Development

This next section is a shortened version of the excellent tutorial on [Defensive programming](https://swc-osg-workshop.github.io/2017-05-17-JLAB/novice/python/05-defensive.html) at Software Carpentry.

Let's assume we want to write a function that calculates the overlap in two or more timeseries, like in the figure below
![overlapping_ranges](https://swc-osg-workshop.github.io/2017-05-17-JLAB/novice/python/img/python-overlapping-ranges.svg)
(Figure from https://swc-osg-workshop.github.io/2017-05-17-JLAB/novice/python/05-defensive.html)

The idea of Test Driven Development is that we will now write the test cases _before_ we write the actual function. 

So for example

In [ ]:
def test_range_overlap():
    assert range_overlap([(-3, 5), (0, 4.5), (-1.5, 2)]) == (0, 2)
    assert range_overlap([(0.0, 1.0)]) == (0.0, 1.0)
    assert range_overlap([(2.0, 3.0), (2.0, 4.0)]) == (2.0, 3.0)
    assert range_overlap([(-200, -150), (-170, -165)]) == (-170, -165)
    assert range_overlap([(0.0, 1.0), (0.0, 2.0), (-1.0, 1.0)]) == (0.0, 1.0)
    assert range_overlap([(0.0, 1.0), (2.0, 4.0)]) is None
    assert range_overlap([(0.0, 1.0), (1.0, 2.0)]) == (1.0, 1.0)  # Note could also have chosen None here
    print('All unit tests passed')

Of course, since we haven't written the function `range_overlap` yet, it makes little sense to call this `test_range_overlap()` function. But it did force us to think about exceptions like what happens if there is no overlap.

This is Test Driven Development. Much more information, including motivation why it's a better way of development, can be found at [the Software Carpentry lesson on Defensive programming](https://swc-osg-workshop.github.io/2017-05-17-JLAB/novice/python/05-defensive.html).

***Exercise: write the function `range_overlap()` that passes all unit tests of the `test_range_overlap()` function above***

In [ ]:
# Answer
def range_overlap(ranges):
    '''Return common overlap among a set of [low, high] ranges.'''
    lowest = -np.inf
    highest = np.inf
    for (low, high) in ranges:
        if high < lowest or low > highest:
            return None
        lowest = max(lowest, low)
        highest = min(highest, high)
    return (lowest, highest)

test_range_overlap()

One thing that can still go wrong in this function is if the `low > high` within a tuple `(low, high`). So we will need to check for that too. In the `test_range_overlap()` function we can add another `assert`, but now one that throws an error.

In [ ]:
def test_range_overlap():
    assert range_overlap([(-3, 5), (0, 4.5), (-1.5, 2)]) == (0, 2)
    assert range_overlap([(0.0, 1.0)]) == (0.0, 1.0)
    assert range_overlap([(2.0, 3.0), (2.0, 4.0)]) == (2.0, 3.0)
    assert range_overlap([(-200, -150), (-170, -165)]) == (-170, -165)
    assert range_overlap([(0.0, 1.0), (0.0, 2.0), (-1.0, 1.0)]) == (0.0, 1.0)
    assert range_overlap([(0.0, 1.0), (2.0, 4.0)]) is None
    assert range_overlap([(0.0, 1.0), (1.0, 2.0)]) == (1.0, 1.0)  # Note could also have chosen None here
    try:
        range_overlap([(4, 3), (3, 4)])
        assert False
    except ValueError:
        pass
    print('All unit tests passed')

***Exercise: Add a `raise ValueError()` to your `range_overlap` function***

In [ ]:
# Answer

def range_overlap(ranges):
    '''Return common overlap among a set of [low, high] ranges.'''
    lowest = -np.inf
    highest = np.inf
    for (low, high) in ranges:
        if high < low:
            raise ValueError('high < low for at least one of the input tuples') 
        if high < lowest or low > highest:
            return None
        lowest = max(lowest, low)
        highest = min(highest, high)
    return (lowest, highest)

test_range_overlap()

### Code developing according to XKCD

(because no coding lesson is complete without an XKCD commic to wrap it up)

![](https://imgs.xkcd.com/comics/the_general_problem.png)
https://xkcd.com/974/

![](https://imgs.xkcd.com/comics/good_code.png)
https://xkcd.com/844/

![](https://imgs.xkcd.com/comics/automation.png)
https://xkcd.com/1319/